In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import string
from spacy.lang.en.stop_words import STOP_WORDS
import spacy
from sklearn.svm import LinearSVC
from google_trans_new import google_translator

# Training data

In [2]:
data = pd.read_csv('amazon_data.txt', sep='\t', header=None)

In [3]:
data.columns = ["Review", "Sentiment"]
data.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [4]:
data["Sentiment"].value_counts() # checking data

1    500
0    500
Name: Sentiment, dtype: int64

In [5]:
data.isnull().sum() # checking for null values

Review       0
Sentiment    0
dtype: int64

In [6]:
## helpers for data cleaning
punctuation = string.punctuation
stop_words = list(STOP_WORDS)
nlp = spacy.load("en_core_web_sm")
numbers = string.digits
# print(stop_words)
## function that cleans input text
def cleaning_function(input_text):
    text = nlp(input_text)
    tokens = []
    for token in text:
        temp = token.lemma_.lower()
        tokens.append(temp)

    cleaned_tokens = []
    for token in tokens:
        if token not in stop_words and token not in punctuation and token not in numbers:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [7]:
## test
cleaned_text = cleaning_function(data["Review"][4])
print(cleaned_text)

['mic', 'great']


In [8]:
X = data["Review"]
y = data["Sentiment"]

In [9]:
## SVC using tfidf (bag of words)

tfidf = TfidfVectorizer(tokenizer = cleaning_function)
classifier = LinearSVC()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=50)
SVC_clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])
SVC_clf.fit(X_train, y_train)
y_pred = SVC_clf.predict(X_test)

print("Classification Report for SVC:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix for SVC:")
print(confusion_matrix(y_test, y_pred))

Classification Report for SVC:
              precision    recall  f1-score   support

           0       0.75      0.82      0.79       154
           1       0.79      0.72      0.75       146

    accuracy                           0.77       300
   macro avg       0.77      0.77      0.77       300
weighted avg       0.77      0.77      0.77       300

Confusion Matrix for SVC:
[[126  28]
 [ 41 105]]


In [10]:
def predict_func(string):
    temp = SVC_clf.predict([string])
    if (temp[0]==0):
        return("Negative Review")
    else:
        return("Positive Review")

In [11]:
user_input_text = input("Enter review: ")
user_input_pred = predict_func(user_input_text)
print("-->",user_input_pred)

Enter review: Good product! I recommend for everyone to buy.
--> Positive Review


# Text Scraping

In [12]:
## Add feature: input URL by user and output how many positive and negative comments

In [52]:
from selenium import webdriver
from pathlib import Path 
from bs4 import BeautifulSoup

DRIVER_PATH= str(Path('chromedriver').resolve())

user_input_URL = input("Enter Amazon URL: ")

# print(user_input_URL)

# https://www.amazon.com/Acer-SB220Q-Ultra-Thin-Frame-Monitor/dp/B07CVL2D2S/ref=lp_16225007011_1_10?th=1

Enter Amazon URL: https://www.amazon.com/Acer-SB220Q-Ultra-Thin-Frame-Monitor/dp/B07CVL2D2S/ref=lp_16225007011_1_10?th=1


In [33]:
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get(user_input_URL)
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
comments = soup.find_all('div', {'data-hook':'review-collapsed'})

In [37]:
# print(comments[0].span.text)
# for card in cards:
#     print (card)

comment_arr = []
translator = google_translator()

for comment in comments:
    
    comment_temp = comment.span.text
    
    if(translator.detect(comment_temp)!='en'):
        comment_temp = translator.translate(comment_temp, lang_tgt='en')
    
    comment_arr.append(comment_temp)

In [38]:
print(comment_arr[0])

This monitor is definitely a good value. Does it have superb color and contrast? No. Does it boast the best refresh rate on the market? No. But if you're tight on money, this thing looks and preforms great for the money. It has a Matte screen which does a great job at eliminating glare. The chassis it's enclosed within is absolutely stunning. It features a VGA and and HDMI port. The screen automatically detects signals and turns on when a source is turned on. When the selected source turns off it quickly scans and switches to any other sources. If no connections are available it goes into standby automatically. The power chord is a good 6 or more feet and easy to wrap with half of it being very thin wire. The ON LED is not distracting or too bright. When it's off it looks like a borderless monitor. And unlike what other people say it does have tilt, just not forward or left to right. It stands up straight or leans back. Unfortunately it doesn't have VESA mount holes, but under 100 you 

In [46]:
## PREDICT

pred_arr = []
pos = 0
neg = 0

for comment in comment_arr:
    comment_pred = predict_func(comment)
    if comment_pred=='Positive Review':
        pos+=1
    else:
        neg+=1
    pred_arr.append(comment_pred)
    
print(pred_arr)
print("Total number of positive reviews:", pos)
print("Total number of negative reviews:", neg)

['Positive Review', 'Positive Review', 'Positive Review', 'Negative Review', 'Positive Review', 'Negative Review', 'Positive Review', 'Positive Review', 'Negative Review', 'Positive Review', 'Positive Review', 'Positive Review']
Total number of positive reviews: 9
Total number of negative reviews: 3


In [44]:
print(comment_arr[8])

Exact value for money, you pay for this monitor, greatly a monitor of 6Bits + FRC is not 8 truly bits, this serves for gaming for the quick response you have but not to see good colors, not really It has good colors, in general they look wishes but I can not ask for more for this low price. As soon as angles of vision ... you have to be straight to him, you can not see it anywhere because you will see the bleeding of backlight or the even more unlawned colors, it has a very good glitter, of echo hurts you if you put it to the fullest At night, it is extremely bright. By default my NVIDIA card detects it as a 75Hz monitor, hiding the other modes of 60 Hz 50 Hz 30Hz 25Hz 24 Hz 23.7 Hz. It has good sharpness and has good black but I repeat, there is a backlight bleeding even if it is not so severe and I repeat it is good for gaming not for vivid colors or watching movies with vibrant and pretty colors. What if it bothers me about these panels is that neither on the official page mention t

In [47]:
## Issues
# Language -> translation or limit to only eng - fixed
# Pages -> Load all reviews by going to all reivew pages

In [49]:
all_reviews_button = driver.find_elements_by_xpath('//*[@id="cr-pagination-footer-0"]/a')[0]
all_reviews_button.click()

In [ ]:
translator = google_translator()

all_pages_reached = False

comment_arr = []

while not all_pages_reached: # iterate through all comments in all pages
    comments = soup.find_all('div', {'data-hook':'review-collapsed'})
    
    for comment in comments:
    
        comment_temp = comment.span.text
    
        if(translator.detect(comment_temp)!='en'):
            print("b4:",comment_temp)
            comment_temp = translator.translate(comment_temp, lang_tgt='en')
            print("after:",comment_temp)
            
    comment_arr.append(comment_temp)
    